<a href="https://colab.research.google.com/github/tejeshb/Medley-generator/blob/main/Medley_Creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install spleeter, ffmpeg, pydub


In [1]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [2]:
!pip install spleeter

     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 184kB 9.6MB/s 
     |████████████████████████████████| 20.1MB 12.7MB/s 
     |████████████████████████████████| 320.4MB 49kB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 10.5MB 59.3MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 460kB 58.3MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
  Created wheel for librosa: filename=librosa-0.8.0-cp36-none-any.whl size=201375 sha256=cfc98753a9d873054e7e34bdb4dbc9f648af9e0684e2cc50daa7340104bb3d13
  Stored in directory: /root/.cache/pip/wheels/ee/10/1e/382bb4369e189938d5c02e06d10c651817da8d485bfd1647c9
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl 

In [1]:
pip install pydub

In [2]:
#import libraries
from IPython.display import Audio
import librosa
import librosa.display
import pydub
from pydub import AudioSegment
import soundfile as sf

In [3]:
#Know how spleeter can be used
!spleeter separate --help

Usage: spleeter separate [OPTIONS] FILES...

  Separate audio file(s)

Arguments:
  FILES...  List of input audio file path
            [required]


Options:
  -i, --inputs TEXT               (DEPRECATED)
                                  placeholder for
                                  deprecated input
                                  option

  -a, --adapter TEXT              Name of the
                                  audio adapter to
                                  use for audio
                                  I/O  [default: s
                                  pleeter.audio.ff
                                  mpeg.FFMPEGProce
                                  ssAudioAdapter]

  -b, --bitrate TEXT              Audio bitrate to
                                  be used for the
                                  separated output
                                  [default: 128k]

  -c, --codec [wav|mp3|ogg|m4a|wma|flac]
                                  Audio codec to
          

Separate voice from songs

In [4]:
!spleeter separate song1.mp3 -d 60 -o output/

INFO:spleeter:Downloading model archive https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz
INFO:spleeter:Validating archive checksum
INFO:spleeter:Extracting downloaded 2stems archive
INFO:spleeter:2stems model file(s) extracted
INFO:spleeter:File output/song1/accompaniment.wav written succesfully
INFO:spleeter:File output/song1/vocals.wav written succesfully


In [5]:
!spleeter separate song2.mp3 -d 60 -o output/

INFO:spleeter:File output/song2/vocals.wav written succesfully
INFO:spleeter:File output/song2/accompaniment.wav written succesfully


In [6]:
!spleeter separate song3.mp3 -d 60 -o output/

INFO:spleeter:File output/song3/vocals.wav written succesfully
INFO:spleeter:File output/song3/accompaniment.wav written succesfully


In [7]:
#path of voice files
songs_path = ["/content/output/song1/vocals.wav","/content/output/song2/vocals.wav","/content/output/song3/vocals.wav"]

In [8]:
#declare lists for processing
y_all = []
sr_all = []
tr_all = []

In [9]:
#Extract the y and sr
for song in songs_path:
  y, sr = librosa.load(song, duration=60)
  y_all.append(y)
  sr_all.append(sr)

In [10]:
#Get the voice part that is over 40 decibles, idea is to remove silence from the files
for x in y_all:
  tr_all.append(librosa.effects.split(y=x, frame_length=8000, top_db=40))

Get start and end times of the non silenced parts to comapre and pick best snippet

In [11]:
duration = []
start_time = []
end_time = []
dur = []
c = 0
for tr in tr_all: 
  start_time.append([])
  end_time.append([])
  duration.append([])

  for i in tr:
    st = ((i[1]/sr_all[c]) - (i[0]/sr_all[c]))
    start_time[c].append(str(i[0]/sr_all[c]))
    end_time[c].append(str(i[1]/sr_all[c]))
    duration[c].append(st)
  c= c+1


In [12]:
song_offset = []
song_snip_duration_index = []
song_snip_duration = []

In [13]:
for d in duration:
   print(d)
   song_snip_duration_index.append(d.index(max(d)))
   song_snip_duration.append(max(d))

[3.5526530612244898, 5.4334693877551015, 3.3901133786848057, 4.876190476190475, 3.5526530612244898, 0.3482993197278894, 9.380861678004536, 0.7430385487528355, 6.222947845804988, 3.8312925170068084, 0.37151927437641774, 5.178049886621309, 3.343310657596369]
[2.7863945578231295, 0.3482993197278912, 0.37151927437641774, 3.78485260770975, 0.3947392290249425, 6.98920634920635, 0.3715192743764142, 2.623854875283449, 0.37151927437641774, 0.3947392290249425, 3.0650340136054446, 0.3250793650793682, 3.5526530612244898, 3.320453514739228, 0.3018594104308434, 0.023219954648524777, 3.9009523809523827, 0.3947392290249425, 6.083265306122449]
[5.596009070294785, 6.246167800453516, 9.589841269841271, 7.314285714285717, 6.548027210884349, 10.17034013605442, 0.9284353741496574]


In [14]:
for i,s in enumerate(start_time):
  song_offset.append(s[song_snip_duration_index[i]])

In [15]:
song_offset

['26.79582766439909', '15.650249433106575', '45.69687074829932']

In [16]:
for n,new_song in enumerate(songs_path):
  new_y, new_sr = librosa.load( new_song, offset=float(song_offset[n]),duration=song_snip_duration[n])
  sf.write("new_song" + str(n) + ".wav", new_y, new_sr)


In [ ]:
#join audio files

In [ ]:
sound1 = AudioSegment.from_wav("/content/new_song0.wav")
sound2 = AudioSegment.from_wav("/content/new_song1.wav")
sound3 = AudioSegment.from_wav("/content/new_song2.wav")

combined_sounds = sound1 + sound2 + sound3
combined_sounds.export("all_voices.wav", format="wav")

In [ ]:
Audio('all_voices.wav')